In [1]:
import random 
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.by import By 
from time import sleep 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager 
import os 
import dotenv
import pandas as pd 
from bs4 import BeautifulSoup
import requests 

In [2]:
dotenv.load_dotenv('develop.env')

True

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
login_page = "https://login.dlsu.idm.oclc.org/login?qurl=https://cdasiaonline.com%2fl%2fee17a146%2fsearch%3fyear_end%3d2022%26year_start%3d2022"

driver.get(login_page)
driver.maximize_window() 

options = Options() 
options.add_argument("--headless")
options.add_argument("--start-maximized") 
options.add_argument("--disable-notifications")
options.add_argument("--incognito")
sleep(3) 
# sleep(3)

username = driver.find_element_by_name('user') 
username.send_keys(os.environ['CDA_UN'])
sleep(1)

password = driver.find_element_by_name('pass')
password.send_keys(os.environ['CDA_PW'])
sleep(1)

submit_button = driver.find_element_by_name('login')
submit_button.click()
sleep(3)

continue_button = driver.find_element_by_class_name('btn-card-submit')
continue_button.click()

# scraping the page 
cases = driver.find_elements_by_class_name('i-menu-newtab')
data_list = [] 
for case in cases: 
    td = case.find_elements_by_tag_name('td')
    

    ref_num = td[0].text
    case_name = td[1].text 
    judge = td[2].text 
    date = td[4].text 
    url = case.get_attribute('data-href')

    data = {'ref_num': ref_num, 
            'case': case_name, 
            'judge': judge, 
            'date':date, 
            'url':url} 

    data_list.append(data)

# doc-view-container js-content

def scrape_cases(url): 
    sleep(1)
    # try:
    driver.get(url)
    doc = driver.find_element_by_class_name('doc-view-container')
    
    return doc.text.strip() 
    # 36 /124 have broken links -- have been designated null -- add to documentation 
    # except: 
    #     return None


# exporting to a dataframe & csv
df = pd.DataFrame(data_list)
df['body_text'] = df['url'].apply(lambda x: scrape_cases(x))
df.to_csv('cases_data.csv', index=False)
df

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\hxwwo\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\hxwwo\AppData\Local\Temp\ipykernel_15468\1838590715.py:15: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  username = driver.find_element_by_name('user')
C:\Users\hxwwo\AppData\Local\Temp\ipykernel_15468\1838590715.py:19: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  password = driver.find_element_by_name('pass')
C:\Users\hxwwo\AppData\Local\Temp\ipykernel_15468\1838590715.py:23: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  submit_button = driver.find_element_by_name('login')
C:\Users\hxwwo\AppData\Local\Temp\ip

,ref_num,case,judge,date,url,body_text
0,G.R. No. 214590,Estate of Rodriguez v. Republic,"HERNANDO, J :",2022-04-27,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"SECOND DIVISION\n[G.R. No. 214590. April 27, 2..."
1,G.R. No. 208258,Carullo-Padua v. Padua,"HERNANDO, J :",2022-04-27,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"SECOND DIVISION\n[G.R. No. 208258. April 27, 2..."
2,G.R. Nos. 245931-32,People v. Partisala,"HERNANDO, J :",2022-04-25,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,SECOND DIVISION\n[G.R. Nos. 245931-32. April 2...
3,G.R. No. 217119,Systems and Plan Integrator and Development Co...,"HERNANDO, J :",2022-04-25,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"SECOND DIVISION\n[G.R. No. 217119. April 25, 2..."
4,G.R. No. 215175,Aljem's Credit Investors Corp. v. Spouses Baut...,"HERNANDO, J :",2022-04-25,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"SECOND DIVISION\n[G.R. No. 215175. April 25, 2..."
5,G.R. No. 213888,Heirs of Borras v. Heirs of Borras,"HERNANDO, J :",2022-04-25,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"SECOND DIVISION\n[G.R. No. 213888. April 25, 2..."
6,G.R. No. 219755,Wahing v. Spouses Daguio,"LEONEN, J :",2022-04-18,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"THIRD DIVISION\n[G.R. No. 219755. April 18, 20..."
7,G.R. No. 258947,Commissioner of Internal Revenue v. Court of T...,"CAGUIOA, J :",2022-03-29,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"FIRST DIVISION\n[G.R. No. 258947. March 29, 20..."
8,G.R. Nos. 240187-88,"Buenaflor v. Federated Distributors, Inc.","J.Y. LOPEZ, J :",2022-03-28,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,THIRD DIVISION\n[G.R. Nos. 240187-88. March 28...
9,G.R. No. 216771,Ubalde v. Morales,"J.Y. LOPEZ, J :",2022-03-28,https://cdasiaonline-com.dlsu.idm.oclc.org/jur...,"THIRD DIVISION\n[G.R. No. 216771. March 28, 20..."
